In [ ]:
# INICIALMENTE A IDEIA ERA DE SE USAR UM MODELO DE REGRESSÃO LINEAR SIMPLES PARA PREVER PREÇOS FUTUROS
# PORÉM, COMO AINDA EXISTEM OUTLIERS QUE PASSARAM A FILTRAGEM, É NECESSÁRIO O USO DE UM MODELO MAIS ROBUSTO QUE NÃO SE DEIXE SER AFETADO POR OUTLIERS
# COMO NÃO TENHO TANTA EXPERIÊNCIA COM O USO DE MACHINE LEARNING, IREI CRIAR UM MODELO DE REGRESSÃO RANDOM FOREST
# ESSE MODELO É MAIS ROBUSTO QUE O MODELO DE REGRESSÃO LINEAR SIMPLES, PORÉM ELE AINDA É FALHO A OUTLIERS
# PORTANTO OS RESULTADOS PODEM NÃO SER OS MAIS CONFIÁVEIS, SENDO A NECESSIDADE DE SE UTILIAR UM MODELO COM MAIOR ROBUSTEZ A OUTLIERS COMO O HUBER REGRESSION

**MODELO DE REGRESSÃO RANDOM FOREST**

---



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
df_filtered_final = pd.read_csv("df_price_train.csv")

In [ ]:
# Separando as colunas em numéricas e categóricas
cols_num = ["minimo_noites", "numero_de_reviews", "reviews_por_mes",
            "calculado_host_listings_count", "disponibilidade_365"]
cols_ctg = ["bairro_group", "bairro", "room_type"]

# Criando dummies para variáveis categóricas
df_encoded = pd.get_dummies(df_filtered_final, columns=cols_ctg, drop_first=True)

# Separar x e y
x = df_encoded.drop(columns=["price"])
y = df_encoded["price"]

# Normalizar os dados numéricos
scaler = StandardScaler()
x[cols_num] = scaler.fit_transform(x[cols_num])

In [ ]:
# Dividir em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Criar e treinar o modelo
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = rf.predict(x_test)

# Avaliar o modelo
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibindo os resultados
print(f"Erro Absoluto Médio (MAE): {mae:.2f}")
print(f"Erro quadrático médio (MSE): {mse:.2f}")
print(f"Coeficiente de determinação (R²): {r2:.2f}")

In [ ]:
# Criando um DataFrame com os dados do novo imóvel
new_house_price = pd.DataFrame([{
    "bairro_group": "Manhattan",
    "bairro": "Midtown",
    "room_type": "Entire home/apt",
    "minimo_noites": 1,
    "numero_de_reviews": 45,
    "reviews_por_mes": 0.38,
    "calculado_host_listings_count": 2,
    "disponibilidade_365": 355
}])

# Aplicando o mesmo pré-processamento (One-Hot Encoding)
new_house_price = pd.get_dummies(new_house_price)
new_house_price = new_house_price.reindex(columns=x.columns, fill_value=0)

# Fazer a previsão
preco_estimado = rf.predict(new_house_price)[0]
print(f"Preço estimado: ${preco_estimado:.2f}")

**MODELO DE REGRESSÃO LINEAR SIMPLES PARA COMPARAÇÃO**

---



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# Separando as colunas em numéricas e categóricas
cols_num = ["minimo_noites", "numero_de_reviews", "reviews_por_mes",
            "calculado_host_listings_count", "disponibilidade_365"]
cols_ctg = ["bairro_group", "bairro", "room_type"]

# Criando dummies para variáveis categóricas
df_encoded = pd.get_dummies(df_filtered_final, columns=cols_ctg, drop_first=True)

# Separar x e y
x = df_encoded.drop(columns=["price"])
y = df_encoded["price"]

# Normalizar os dados numéricos
scaler = StandardScaler()
x[cols_num] = scaler.fit_transform(x[cols_num])

In [ ]:
# Dividir em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Criar e treinar o modelo de Regressão Linear
lr = LinearRegression()
lr.fit(x_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = lr.predict(x_test)

# Avaliar o modelo
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibindo os resultados
print(f"Erro Absoluto Médio (MAE): {mae:.2f}")
print(f"Erro quadrático médio (MSE): {mse:.2f}")
print(f"Coeficiente de determinação (R²): {r2:.2f}")

In [ ]:
# Criando um DataFrame com os dados do novo imóvel
new_house_price = pd.DataFrame([{
    "bairro_group": "Manhattan",
    "bairro": "Midtown",
    "room_type": "Entire home/apt",
    "minimo_noites": 1,
    "numero_de_reviews": 45,
    "reviews_por_mes": 0.38,
    "calculado_host_listings_count": 2,
    "disponibilidade_365": 355
}])

# Aplicando o mesmo pré-processamento (One-Hot Encoding)
new_house_price = pd.get_dummies(new_house_price)
new_house_price = new_house_price.reindex(columns=x.columns, fill_value=0)

# Normalizar as variáveis numéricas (usando o mesmo scaler do treino)
new_house_price[cols_num] = scaler.transform(new_house_price[cols_num])

# Fazer a previsão usando o modelo de Regressão Linear
preco_estimado = lr.predict(new_house_price)[0]
print(f"Preço estimado: ${preco_estimado:.2f}")

**CONCLUSÃO DOS TESTES**

---



In [ ]:
# MESMO QUE O DATAFRAME AINDA POSSUA OUTLIERS QUE ALTEREM O RESULTADO FINAL DA PREVISÃO
# É POSSÍVEL VER QUE O MODELO RANDOM FOREST POSSUI MAIS ROBUSTES AOS OUTLIERS SE COMPARADO AO MODELO DE REGRESSÃO LINEAR MAIS SIMPLES